<a href="https://colab.research.google.com/github/Daesung-Jung/baseball_pitchdesign/blob/main/execute_gpu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1. 환경설정 확인
>  GPU 확인, 
깃허브 private clone 

In [ ]:
#코랩 GPU 확인
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

 # 추가 메모리 확인 
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('To enable a high-RAM runtime, select the Runtime > "Change runtime type"')
  print('menu, and then select High-RAM in the Runtime shape dropdown. Then, ')
  print('re-execute this cell.')
else:
  print('You are using a high-RAM runtime!') 

In [ ]:
#import torch
#torch.cuda.empty_cache()

In [ ]:
!ssh-keygen -t rsa -b 4096 -C "shtnrgudsla@naver.com"

In [ ]:
!ssh-keyscan -t rsa github.com >> ~/.ssh/known_hosts

> 아래 나오는 키 복사해서 깃허브 계정 내 SSH~ 어쩌고 내용에 저장. title은 암거나.

In [ ]:
!cat /root/.ssh/id_rsa.pub

In [ ]:
!ssh -T git@github.com

!git config --global user.email "shtnrgudsla@naver.com"
!git config --global user.name "d!"



# 2. git clone 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/
# 주소대신 ssh copy 
!git clone git@github.com:Daesung-Jung/baseball_pitchdesign.git
# 존재한다면 최신화
%cd /content/drive/MyDrive/baseball_pitchdesign/
!git pull
#필요한 패키지 설치
%cd /content/drive/MyDrive/baseball_pitchdesign/
!pip install -r requirements.txt

In [ ]:
import yaml
%cd /
from glob import glob

img_list = glob('/content/drive/MyDrive/baseball_pitchdesign/images/*.jpg')
lab_list = glob('/content/drive/MyDrive/baseball_pitchdesign/labels/*.txt')

print(len(img_list))
print(len(lab_list))

In [ ]:
from sklearn.model_selection import train_test_split

train_img_list, val_img_list = train_test_split(img_list, test_size=0.2, random_state=2000)

print(len(train_img_list), len(val_img_list))

In [ ]:
with open('/content/drive/MyDrive/baseball_pitchdesign/train.txt', 'w') as f:
  f.write('\n'.join(train_img_list) + '\n')

with open('/content/drive/MyDrive/baseball_pitchdesign/val.txt', 'w') as f:
  f.write('\n'.join(val_img_list) + '\n')

In [ ]:
import yaml

with open('/content/drive/MyDrive/baseball_pitchdesign/data.yaml', 'r') as f:
  data = yaml.load(f)

print(data)
data['train'] = '/content/drive/MyDrive/baseball_pitchdesign/train.txt'
data['val'] = '/content/drive/MyDrive/baseball_pitchdesign/val.txt'

print(data['val'])
with open('/content/drive/MyDrive/baseball_pitchdesign/data.yaml', 'w') as f:
  yaml.dump(data, f)

print(data)

# 3. 학습하기 

In [ ]:
%cd /content/drive/MyDrive/baseball_pitchdesign/yolov5/
!python train.py --img 416 --batch 32 --epochs 600 --data /content/drive/MyDrive/baseball_pitchdesign/data.yaml --cfg ./models/yolov5s.yaml --weights yolov5s.pt --name elbow_bg_all_32_600

# 4. 결과 비교

In [ ]:
%load_ext tensorboard
%tensorboard --logdir /content/drive/MyDrive/baseball_pitchdesign/yolov5/runs/

# 5. 실제 데이터 넣기


In [ ]:
%cd /content/drive/MyDrive/baseball_pitchdesign/yolov5/
!python detect.py --weight /content/drive/MyDrive/baseball_pitchdesign/weight/elbow_bg_all_plus_600.pt --img 416 --conf 0.8 --source /content/drive/MyDrive/dataset/20210624NCLT02021-2.mp4 --nosave

#6. GCP Storage to Colab

In [ ]:
#접근, 인증필요함 
from google.colab import auth
auth.authenticate_user()


'''
$ gsutil list                           # 나의 버킷 리스트 보기
$ gsutil ls -r gs://버킷이름             # 버킷 안에 들어있는 파일 확인
$ gsutil du -s gs://버킷이름             # 버킷 용량 확인
$ gsutil mb gs://버킷이름                # 버킷 생성
$ gsutil rb gs://버킷이름                # 버킷 삭제
$ gsutil cp 로컬 파일 위치 gs://버킷이름   # 로컬 -> 버킷 복사
$ gsutil cp gs://버킷이름 로컬 파일 위치   # 버킷 -> 로컬 복사
$ gsutil mv 로컬 파일 위치 gs://버킷이름   # 로컬 -> 버킷 이동
$ gsutil mv gs://버킷이름 로컬 파일 위치   # 버킷 -> 로컬 이동
$ gsutil rm gs://버킷이름/파일이름        # 파일 삭제
$ gsutil ls -L gs://버킷이름/파일이름     # 파일 정보 보기
'''

In [ ]:
#구글 클라우드 SDK 설치 
%cd /content/drive/MyDrive/
!curl https://sdk.cloud.google.com | bash

In [ ]:
#실행
!gcloud init

In [ ]:
#Download file from Cloud Storage to Google Colab
!gsutil cp gs://diego/2/20210414KTOB02021-1.mp4 .
#24초 걸림 (2.6기가)


#Upload file from Google Colab to Cloud
#!gsutil cp 20210624NCLT02021-1.mp4 gs://diego/example/

In [ ]:
#GCP에서 구글 드라이브로 (45초)
!gsutil cp gs://diego/1/20210410KTSS02021-2.mp4 /content/drive/MyDrive/20210414KTOB02021-1.mp4
!gsutil cp gs://diego/1/20210410KTSS02021-2.mp4 /content/drive/MyDrive/20210414KTOB02021-2.mp4
!gsutil cp gs://diego/1/20210410KTSS02021-2.mp4 /content/drive/MyDrive/20210414KTOB02021-3.mp4
!gsutil cp gs://diego/1/20210410KTSS02021-2.mp4 /content/drive/MyDrive/20210414KTOB02021-4.mp4

In [ ]:
#실제 진행 
!python /content/drive/MyDrive/baseball_pitchdesign/yolov5/detect.py --weight /content/drive/MyDrive/baseball_pitchdesign/weight/elbow_bg_all_plus_600.pt --img 416 --conf 0.9 --source 20210414KTOB02021-1.mp4 --nosave

In [ ]:
%cd ..